# Matrix example
This notebook roughly follows the matrix example at https://www.gurobi.com/documentation/9.0/quickstart_windows/py_python_matrix_example.html


This example formulates and solves the following simple MIP model
using the matrix API:
```
maximize
    x +   y + 2 z
subject to
    x + 2 y + 3 z <= 4
    x +   y       >= 1
    x, y, z binary
```

In [19]:
import numpy as np
import scipy.sparse as sp
import gurobipy as gp
from gurobipy import GRB

In [20]:
m = gp.Model("matrix1")
x = m.addMVar(shape=3, vtype=GRB.BINARY, name="x") # add the decision variables

In [21]:
obj = np.array([1.0, 1.0, 2.0]) # define the factors for the objective function
m.setObjective(obj @ x, GRB.MAXIMIZE) # set the objective as the product of the variables and their factors (@ operator is for matrix mulitplicaiton/dot product)
tmp1=np.array([1,2,3])

In [22]:
data = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])
A = sp.csr_matrix((data, (row, col)), shape=(2,3)) # create a compressed sparse row-based matrix for the constraints
print(A)

  (0, 0)	1.0
  (0, 1)	2.0
  (0, 2)	3.0
  (1, 0)	-1.0
  (1, 1)	-1.0


In [23]:
print(A.todense()) # print the dense version of the sparse matrix

[[ 1.  2.  3.]
 [-1. -1.  0.]]


Note the negative values of the second row (-1, -1). The constraint however uses positive values (`x + y >= 1`). Flipping the signs transforms it into the required 'less or equal' constraint.

In [24]:
rhs = np.array([4.0, -1.0])  # right hand sides of the constraints
m.addConstr(A @ x <= rhs, name='c')  # add the constraints to the model
m.update()  # is required to print the correct status of the model. Is also called when model is solved
m.printStats()


Statistics for model matrix1 :
  Linear constraint matrix    : 2 Constrs, 3 Vars, 5 NZs
  Variable types              : 0 Continuous, 3 Integer (3 Binary)
  Matrix coefficient range    : [ 1, 3 ]
  Objective coefficient range : [ 1, 2 ]
  Variable bound range        : [ 1, 1 ]
  RHS coefficient range       : [ 1, 4 ]


In [25]:
m.optimize()  # solve the model

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x6d8be88c
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [26]:
print(x.varName)
print(x.X)
print(f'Obj:{m.ObjVal}')

['x[0]' 'x[1]' 'x[2]']
[1. 0. 1.]
Obj:3.0
